## 7회 실기시험 유형파악 - 3유형

## 제3유형 문제파악

### 1. 로지스틱 회귀분석(15점, 각 소문제당 5점)

- 주요키워드 : odds ratio, residual deviance, 오분류율, 모델링, 성능평가

- 1-1) 종속변수(Y)를 A변수로 분석했을 때, odds ratio는? (데이터셋은 총 300개이며, 문제에서 지정한 210개로 모델링함)
- 1-2) 종속변수(Y)를 n개의 변수로 분석했을 때, residual deviance를 구하시오. (데이터셋은 총 300개이며, 문제에서 지정한 210개로 모델링함)
- 1-3) 문제1)에서 구한 모델에 나머지 90개 데이터를 적용하여 오분류율을 구하시오.

### 2. 다중선형회귀분석(15점, 각 소문제당 5점)

- 주요키워드 : 상관계수, Rsq, p-value, 모델링, 성능평가

- 2-1) 특정 변수와 가장 큰 상관관계를 갖는 변수의 상관계수를 구하시오.
- 2-2) 다중선형회귀 모델링 후 결정계수(Rsq) 구하기
- 2-3) 문제 2-2에서 구한 회귀모델에서 p-value가 가장 큰 변수의 p-value값을 구하시오.

## 제 3유형 실전 예제풀이

### 1. 로지스틱 회귀분석(15점, 각 소문제당 5점)
- 1-1) 종속변수(Y)를 A변수로 분석했을 때, odds ratio는? (데이터셋은 총 300개이며, 문제에서 지정한 210개로 모델링함)
- 1-2) 종속변수(Y)를 n개의 변수로 분석했을 때, residual deviance를 구하시오. (데이터셋은 총 300개이며, 문제에서 지정한 210개로 모델링함)
- 1-3) 문제1)에서 구한 모델에 나머지 90개 데이터를 적용하여 오분류율을 구하시오.

### 예제문제 3유형-1번
- survived	를	종속변수(Y),	sex,sibsp,fare	변수를	독립변수(X)로	분석했을	때	아래	질문에	답하시오.
- (단,	총	891개의	데이터에서	초기	500개	데이터로	학습,	391개	데이터로	평가	하시오)	
- (단,	모든	답은	반올림하여	소수점	둘째자리까지	구하시오)
- 1.	초기	500개	데이터로	분석시	sibsp	변수의	odds	ratio는?
- 2.	초기	500개	데이터로	분석시	residual	deviance는?
- 3.	나머지	391개	데이터	적용시	오분류율은?

In [1]:
# 데이터 생성
import pandas as pd
import numpy as np
import seaborn as sns
df = sns.load_dataset('titanic')

df = df[['survived','sex','sibsp','fare']]
# sex:성별, sibsp:탑승한 부모 및 자녀 수, fare:요금
df['sex'] = df['sex'].map({'female':1, 'male':0})
df

,survived,sex,sibsp,fare
0,0,0,1,7.2500
1,1,1,1,71.2833
2,1,1,0,7.9250
3,1,1,1,53.1000
4,0,0,0,8.0500
...,...,...,...,...
886,0,0,0,13.0000
887,1,1,0,30.0000
888,0,1,1,23.4500
889,1,0,0,30.0000


In [2]:
# 데이터셋 구성(초기 500개 데이터로 학습, 391개로 평가)
#print(df.shape)

# train/test 데이터셋 구성
train = df.loc[0:499]
test = df.loc[500:]

x_train = train[['sex','sibsp','fare']]
y_train = train['survived']
x_test = test[['sex','sibsp','fare']]
y_test = test['survived']

In [3]:
# 모델링
from sklearn.linear_model import LogisticRegression
# 반드시 penalty=None으로 입력할 것, default='l2'
model_sk = LogisticRegression(penalty=None)
# 실기체험환경 사이킷런 0.24.2버전 : (penalty='none')
model_sk.fit(x_train, y_train)

LogisticRegression(penalty=None)

In [4]:
# 로지스틱회귀분석 관련 지표 출력
# 회귀계수 출력 : model.coef_
print(np.round(model_sk.coef_, 4))
print(np.round(model_sk.coef_[0,0], 4))
print(np.round(model_sk.coef_[0,1], 4))
print(np.round(model_sk.coef_[0,2], 4))

[[ 2.7284 -0.3799  0.0094]]
2.7284
-0.3799
0.0094


In [5]:
# 회귀계수(절편) : model.intercept_
print(np.round(model_sk.intercept_, 4))

[-1.6815]


결과 : ln(P/1-P) = -1.6815 + 2.7284sex - 0.3799sibsp + 0.0094fare

In [7]:
# 1. odds ratio 구하기
#	로지스틱	회귀모형에서	sibsp	변수가	한단위	증가할	때	생존할	오즈가	몇	배	증가?
# exp(b2)
result = np.exp(model_sk.coef_[0,1])
print(round(result, 2))

# 해석 : sibsp 변수가 한 단위 증가할 때 생존할 오즈가 0.68배 증가한다.
# 생존할 오즈가 32% 감소한다. (생존할 확률이 감소한다)
# 답 : odds ratio 는 0.68

0.68


(정답) 1. sibsp 변수 odds ratio : 0.68

In [8]:
# 2. residual deviance(잔차 이탈도) 구하기
# residual deviance = cross entropy(log loss) * 2
# (주의 : log_loss() 옵션에 normalize = False 설정할 것!)
from sklearn.metrics import log_loss
# 얻게 되는 모델의 이탈도 값 구하기
y_pred_proba = model_sk.predict_proba(x_train) # 주의 : 확률값을 구해야 함
logloss = log_loss(y_train, y_pred_proba, normalize=False)
# 'normalize = true' : if true, return the mean loss per sample.
# Otherwise, return the sum of the per-sample losses.

print(logloss)
print(round(logloss*2, 2))

238.5369401214462
477.07


(정답) 2. residual deviance : 477.07

In [9]:
# 3. 성능평가(정확도와 오분류율)
# 모델을 사용하여 테스트 데이터 예측
y_pred = model_sk.predict(x_test)

# 모델 성능 평가(accuracy)
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, y_pred)

In [10]:
# 정확도(Accuracy)
print(acc)
# 오분류율(1-정확도)
print(round(1-acc, 2))

0.782608695652174
0.22


(정답) 3. 오분류율 : 0.22

### 2. statsmodels 라이브러리 사용

In [11]:
# 모델링 (Logit 함수)
import statsmodels.api as sm

x_train = sm.add_constant(x_train) # 주의 : 상수항 추가해줘야 함
model_logit = sm.Logit(y_train, x_train).fit() # 주의할 것 : y,x순으로 입력
summary = model_logit.summary()
print(summary)

Optimization terminated successfully.
         Current function value: 0.477074
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:               survived   No. Observations:                  500
Model:                          Logit   Df Residuals:                      496
Method:                           MLE   Df Model:                            3
Date:                Tue, 04 Jun 2024   Pseudo R-squ.:                  0.2847
Time:                        13:11:12   Log-Likelihood:                -238.54
converged:                       True   LL-Null:                       -333.46
Covariance Type:            nonrobust   LLR p-value:                 6.573e-41
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.6815      0.174     -9.690      0.000      -2.022      -1.341
sex            2.7284      0.

결과 : ln(P/1-P) = -1.6815 + 2.7284sex - 0.3799sibsp + 0.0094fare (sklearn 결과와 동일)

In [12]:
# exp(b2)를 구하면 된다.
result = np.exp(-0.3799)
print(round(result, 2))

0.68


(정답) 1. sibsp변수의 odds ratio : 0.68

In [13]:
print(model_logit.summary2())

                         Results: Logit
Model:              Logit            Method:           MLE       
Dependent Variable: survived         Pseudo R-squared: 0.285     
Date:               2024-06-04 13:21 AIC:              485.0739  
No. Observations:   500              BIC:              501.9323  
Df Model:           3                Log-Likelihood:   -238.54   
Df Residuals:       496              LL-Null:          -333.46   
Converged:          1.0000           LLR p-value:      6.5725e-41
No. Iterations:     6.0000           Scale:            1.0000    
-------------------------------------------------------------------
           Coef.    Std.Err.      z      P>|z|     [0.025    0.975]
-------------------------------------------------------------------
const     -1.6815     0.1735   -9.6899   0.0000   -2.0216   -1.3414
sex        2.7284     0.2389   11.4229   0.0000    2.2603    3.1966
sibsp     -0.3799     0.1265   -3.0042   0.0027   -0.6278   -0.1321
fare       0.0094     0.

In [14]:
ce = -238.54*-1 # cross entropy = (-)Log-Likelihood
rd = ce*2 # residual deviance = 2 * cross entropy
print(rd) # 반올림해서 소수점 둘째 자리가 다름

477.08


(정답) 2. residual deviance : 477.08 (주의 : 소수점 둘째 자리가 다름)

In [15]:
# 모델링 (GLM 함수)
import statsmodels.api as sm

x_train = sm.add_constant(x_train)
model_glm = sm.GLM(y_train, x_train, family=sm.families.Binomial()).fit()

summary = model_glm.summary()
print(summary)

                 Generalized Linear Model Regression Results                  
Dep. Variable:               survived   No. Observations:                  500
Model:                            GLM   Df Residuals:                      496
Model Family:                Binomial   Df Model:                            3
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -238.54
Date:                Tue, 04 Jun 2024   Deviance:                       477.07
Time:                        14:02:05   Pearson chi2:                     498.
No. Iterations:                     5   Pseudo R-squ. (CS):             0.3159
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.6815      0.174     -9.690      0.0

(정답) 2. residual deviance : 477.07

In [16]:
# 성능평가(정확도와 오분류율) - Logit 함수
import statsmodels.api as sm

x_test = sm.add_constant(x_test) # 주의 : 상수항 추가해줘야 함
model_logit = sm.Logit(y_train, x_train).fit()
# 모델을 사용하여 테스트 데이터 예측(statsmodels 라이브러리)
y_pred_logit = model_logit.predict(x_test)
print(y_pred_logit)

Optimization terminated successfully.
         Current function value: 0.477074
         Iterations 6
500    0.168002
501    0.753984
502    0.753773
503    0.757183
504    0.865585
         ...   
886    0.173796
887    0.790801
888    0.708492
889    0.198027
890    0.166803
Length: 391, dtype: float64


In [17]:
y_pred_logit_class = np.where(y_pred_logit > 0.5, 1, 0)
print(y_pred_logit_class)

[0 1 1 1 1 0 1 0 0 0 0 0 0 1 0 0 1 0 1 0 1 0 0 1 0 0 1 1 0 0 1 0 0 1 1 1 0
 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 1 1 1 0 0 0 0 1 0 0 1 0 0 0 1 0 1
 0 0 1 1 1 0 1 1 0 0 0 1 0 0 0 0 0 1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 1 1
 0 1 0 0 1 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 1 0 0 1 1 0 1 0 0 0
 0 1 0 1 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0
 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 1 0 1 0 0 0 1 0 1 0 1 0 0 0 0 0 1 1 0 0 1 0
 0 0 0 0 1 1 0 1 1 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0
 1 0 0 0 1 0 1 0 1 0 0 0 0 1 0 1 0 0 1 0 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 1 1 0 1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 1 1 0 0
 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 1 1 1 0 1 0 0 0 1 0 0 1 1 0 0 0
 0 1 0 0 1 1 0 0 0 1 1 0 1 0 0 1 0 1 1 0 0]


In [18]:
# 모델 성능 평가 (accuracy)
from sklearn.metrics import accuracy_score
acc_logit = accuracy_score(y_test, y_pred_logit_class)

In [19]:
# 정확도(Accuracy)
print(acc_logit)
# 오분류율(1-정확도)
print(round(1-acc_logit, 2))

0.782608695652174
0.22


(정답) 3. 오분류율 : 0.22

In [20]:
# 성능평가(정확도와 오분류율) - GLM 함수
import statsmodels.api as sm

x_test = sm.add_constant(x_test)
model_glm = sm.GLM(y_train, x_train, family=sm.families.Binomial()).fit()
y_pred_glm = model_glm.predict(x_test)
print(y_pred_glm)

500    0.168002
501    0.753984
502    0.753773
503    0.757183
504    0.865585
         ...   
886    0.173796
887    0.790801
888    0.708492
889    0.198027
890    0.166803
Length: 391, dtype: float64


In [21]:
y_pred_glm_class = np.where(y_pred_glm > 0.5, 1, 0)
print(y_pred_glm_class)

[0 1 1 1 1 0 1 0 0 0 0 0 0 1 0 0 1 0 1 0 1 0 0 1 0 0 1 1 0 0 1 0 0 1 1 1 0
 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 1 1 1 0 0 0 0 1 0 0 1 0 0 0 1 0 1
 0 0 1 1 1 0 1 1 0 0 0 1 0 0 0 0 0 1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 1 1
 0 1 0 0 1 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 1 0 0 1 1 0 1 0 0 0
 0 1 0 1 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0
 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 1 0 1 0 0 0 1 0 1 0 1 0 0 0 0 0 1 1 0 0 1 0
 0 0 0 0 1 1 0 1 1 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0
 1 0 0 0 1 0 1 0 1 0 0 0 0 1 0 1 0 0 1 0 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 1 1 0 1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 1 1 0 0
 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 1 1 1 0 1 0 0 0 1 0 0 1 1 0 0 0
 0 1 0 0 1 1 0 0 0 1 1 0 1 0 0 1 0 1 1 0 0]


In [22]:
# 모델 성능 평가 (accuracy)
from sklearn.metrics import accuracy_score
acc_glm = accuracy_score(y_test, y_pred_glm_class)

In [23]:
# 정확도(Accuracy)
print(acc_glm)
# 오분류율(1-정확도)
print(round(1-acc_glm, 2))

0.782608695652174
0.22


(정답) 3. 오분류율 : 0.22

### 2. 다중선형회귀분석(15점, 각 소문제당 5점)
- 2-1) 특정 변수와 가장 큰 상관관계를 갖는 변수의 상관계수를 구하시오.
- 2-2) 다중선형회귀 모델링 후 결정계수(Rsq) 구하기
- 2-3) 문제 2-2에서 구한 회귀모델에서 p-value가 가장 큰 변수의 p-value값을 구하시오.

### 예제문제 3유형-2번
- 다음은 당뇨병 환자의 질병 진행정도 데이터셋이다.
- target을 종속변수(Y), 나머지를 독립변수(X)로 분석했을 때 아래 질문에 답하시오.
- (단, 모든 답은 반올림하여 소수점 둘째자리까지 구하시오)
- 1. target변수와 가장 큰 상관관계를 갖는 변수의 상관계수를 구하시오.
  2. 다중선형회귀 모델링 후 결정계수(R2 score)를 구하시오.
  3. 문제 2에서 구한 회귀모델에서 p-value가 가장 큰 변수의 p-value 값을 구하시오.

In [24]:
###############		실기환경	복사	영역		###############
 #	데이터	불러오기
import	pandas	as	pd
import	numpy	as	np
#	실기	시험	데이터셋으로	셋팅하기	(수정금지)
from	sklearn.datasets	import	load_diabetes
 #	diabetes	데이터셋	로드
diabetes	=	load_diabetes()
x	=	pd.DataFrame(diabetes.data,	columns=diabetes.feature_names)
y	=	pd.DataFrame(diabetes.target)
y.columns	=	['target']
df = pd.concat([x,y], axis=1)
 ###############		실기환경	복사	영역		###############

- 데이터 정보 :
-	age	:	age	in	years	<br>
-	sex		<br>
-	bmi	:	body	mass	index	<br>	
-	bp		:	average	blood	pressure	<br>	
-	s1		:	tc,	total	serum	cholesterol	<br>	
-	s2		:	ldl,	low-density	lipoproteins	<br>	
-	s3		:	hdl,	high-density	lipoproteins	<br>	
-	s4		:	tch,	total	cholesterol	/	HDL			<br>
-	s5		:	ltg,	possibly	log	of	serum	triglycerides	level	<br>	
-	s6		:	glu,	blood	sugar	level	<br>

In [25]:
df.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135.0


In [26]:
# 1. target 변수와 가장 큰 상관관계를 갖는 변수의 상관계수를 구하시오.
print(df.corr())

             age       sex       bmi        bp        s1        s2        s3  \
age     1.000000  0.173737  0.185085  0.335428  0.260061  0.219243 -0.075181   
sex     0.173737  1.000000  0.088161  0.241010  0.035277  0.142637 -0.379090   
bmi     0.185085  0.088161  1.000000  0.395411  0.249777  0.261170 -0.366811   
bp      0.335428  0.241010  0.395411  1.000000  0.242464  0.185548 -0.178762   
s1      0.260061  0.035277  0.249777  0.242464  1.000000  0.896663  0.051519   
s2      0.219243  0.142637  0.261170  0.185548  0.896663  1.000000 -0.196455   
s3     -0.075181 -0.379090 -0.366811 -0.178762  0.051519 -0.196455  1.000000   
s4      0.203841  0.332115  0.413807  0.257650  0.542207  0.659817 -0.738493   
s5      0.270774  0.149916  0.446157  0.393480  0.515503  0.318357 -0.398577   
s6      0.301731  0.208133  0.388680  0.390430  0.325717  0.290600 -0.273697   
target  0.187889  0.043062  0.586450  0.441482  0.212022  0.174054 -0.394789   

              s4        s5        s6   

In [28]:
# 값을 절대값으로 변환한 후 내림차순으로 정렬하고
# target 변수와 상관계수가 가장 큰 값을 확인한다.
df_corr = df.corr()
df_abs = df_corr.abs()
#print(df_abs)
print(df_abs.sort_values('target', ascending=False))

             age       sex       bmi        bp        s1        s2        s3  \
target  0.187889  0.043062  0.586450  0.441482  0.212022  0.174054  0.394789   
bmi     0.185085  0.088161  1.000000  0.395411  0.249777  0.261170  0.366811   
s5      0.270774  0.149916  0.446157  0.393480  0.515503  0.318357  0.398577   
bp      0.335428  0.241010  0.395411  1.000000  0.242464  0.185548  0.178762   
s4      0.203841  0.332115  0.413807  0.257650  0.542207  0.659817  0.738493   
s3      0.075181  0.379090  0.366811  0.178762  0.051519  0.196455  1.000000   
s6      0.301731  0.208133  0.388680  0.390430  0.325717  0.290600  0.273697   
s1      0.260061  0.035277  0.249777  0.242464  1.000000  0.896663  0.051519   
age     1.000000  0.173737  0.185085  0.335428  0.260061  0.219243  0.075181   
s2      0.219243  0.142637  0.261170  0.185548  0.896663  1.000000  0.196455   
sex     0.173737  1.000000  0.088161  0.241010  0.035277  0.142637  0.379090   

              s4        s5        s6   

In [29]:
# 가장 상관관계가 큰 변수는 bmi 변수
# bmi 변수와 target 변수의 상관계수를 다시 구해보면
print(df.corr())

             age       sex       bmi        bp        s1        s2        s3  \
age     1.000000  0.173737  0.185085  0.335428  0.260061  0.219243 -0.075181   
sex     0.173737  1.000000  0.088161  0.241010  0.035277  0.142637 -0.379090   
bmi     0.185085  0.088161  1.000000  0.395411  0.249777  0.261170 -0.366811   
bp      0.335428  0.241010  0.395411  1.000000  0.242464  0.185548 -0.178762   
s1      0.260061  0.035277  0.249777  0.242464  1.000000  0.896663  0.051519   
s2      0.219243  0.142637  0.261170  0.185548  0.896663  1.000000 -0.196455   
s3     -0.075181 -0.379090 -0.366811 -0.178762  0.051519 -0.196455  1.000000   
s4      0.203841  0.332115  0.413807  0.257650  0.542207  0.659817 -0.738493   
s5      0.270774  0.149916  0.446157  0.393480  0.515503  0.318357 -0.398577   
s6      0.301731  0.208133  0.388680  0.390430  0.325717  0.290600 -0.273697   
target  0.187889  0.043062  0.586450  0.441482  0.212022  0.174054 -0.394789   

              s4        s5        s6   

(정답) 1. target 변수와 가장 큰 상관관계를 갖는 변수의 상관계수 : 0.59

In [30]:
# 다중회귀분석 모델링
# 독립변수와 종속변수 설정
x = df.drop('target', axis=1)
y = df['target']
print(x.head())
print(y.head())

        age       sex       bmi        bp        s1        s2        s3  \
0  0.038076  0.050680  0.061696  0.021872 -0.044223 -0.034821 -0.043401   
1 -0.001882 -0.044642 -0.051474 -0.026328 -0.008449 -0.019163  0.074412   
2  0.085299  0.050680  0.044451 -0.005670 -0.045599 -0.034194 -0.032356   
3 -0.089063 -0.044642 -0.011595 -0.036656  0.012191  0.024991 -0.036038   
4  0.005383 -0.044642 -0.036385  0.021872  0.003935  0.015596  0.008142   

         s4        s5        s6  
0 -0.002592  0.019907 -0.017646  
1 -0.039493 -0.068332 -0.092204  
2 -0.002592  0.002861 -0.025930  
3  0.034309  0.022688 -0.009362  
4 -0.002592 -0.031988 -0.046641  
0    151.0
1     75.0
2    141.0
3    206.0
4    135.0
Name: target, dtype: float64


In [31]:
# 모델링
import statsmodels.api as sm

x = sm.add_constant(x)
model = sm.OLS(y,x).fit()
summary = model.summary()
print(summary)

                            OLS Regression Results                            
Dep. Variable:                 target   R-squared:                       0.518
Model:                            OLS   Adj. R-squared:                  0.507
Method:                 Least Squares   F-statistic:                     46.27
Date:                Tue, 04 Jun 2024   Prob (F-statistic):           3.83e-62
Time:                        14:28:37   Log-Likelihood:                -2386.0
No. Observations:                 442   AIC:                             4794.
Df Residuals:                     431   BIC:                             4839.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        152.1335      2.576     59.061      0.0

(정답) 2. 결정계수(R2 score) : 0.52

(정답) 3. 회귀모델에서 p-value가 가장 큰 변수의 p-value값 : 0.87(age 변수)